In [2]:
import pandas as pd
import numpy as np
import ast
import xmltodict
from cue_scope import load_cue_learner , load_scope_learner
import negtool as negtool
import utils as negutil
from sklearn.externals import joblib
import re

In [ ]:
#key = 
xmlFile = 'post_22118_2.txt.xml'
print 'C:/Users/varun.behl/Documents/test_plain_text_updated/output/output/'+xmlFile
i=0
with open('C:/Users/varun.behl/Documents/test_plain_text_updated/output/output/'+xmlFile.format(i)) as fd:
    doc2 = xmltodict.parse(fd.read())

In [38]:
df_100 = pd.read_csv('data_100_w_pattern_filter.csv')

In [61]:
with open("updated_drugs_v2.txt","r") as f:
        drug = f.read()

In [64]:
drug

'Avelumab|Aldesleukin|Azacitidine|Bcg|Bendamustine|Hydrochloride|Bevacizumab|Bicalutamide|Bortezomib|Cabazitaxel|Cabozantinib|Malate|Capecitabine|Carboplatin|Carfilzomib|Carmustine|Cetuximab|Chlorambucil|Cisplatin|Cisplatin-Vinblastine-Dacarbazine|Cladribine|Cobimetinib|Crizotinib|Dabrafenib|Dacarbazine|Dasatinib|Decitabine|Docetaxel|Doxorubicin|Doxorubicin|Pegylated|Liposomal|Enzalutamide|Everolimus|Exemestane|Fludarabine|Phosphate|Fluorouracil|Fulvestrant|Gemcitabine|Hydroxyurea|Ibrutinib|Idelalisib|Ifosfamide|Imatinib|Interferon|Interferon|Alfa-2A|Interferon|Alfa-2B|Ipilimumab|Irinotecan|Lenalidomide|Lomustine|Melphalan|Mercaptopurine|Methotrexate|Sodium|Nivolumab|Obinutuzumab|Ofatumumab|Olaparib|Olaratumab|Oxaliplatin|Paclitaxel|Palbociclib|Panitumumab|Peginterferon|Peginterferon|Alfa-2A|Peginterferon|Alfa-2B|Pembrolizumab|Pertuzumab|Ramucirumab|Rigvir|Sunitinib|Malate|T-Vec|Tamoxifen|Tamoxifen|Citrate|Temozolomide|Temsirolimus|Thalidomide|Tipiracil|Trametinib|Trifluridine|Vemurafe

### Negtool - Determining the negated indices and words

In [73]:

def isDrugMention(token):
    with open("updated_drugs_v2.txt","r") as f:
        drug = f.read()
    try:
        #print token
        if len(re.findall(r"\b(?=\w)"+token+r"\b(?!\w)",drug, re.IGNORECASE))>=1:
            #print 1
            return 1
    except:
        return 0

In [74]:
def negation_removal(key,count):
    
    xmlFile = 'post'+'_'+str(key)+'.txt.xml'
    #print xmlFile
    
    with open('C:/Users/varun.behl/Documents/test_plain_text_updated/output_from_domino/'+xmlFile) as fd:
        doc2 = xmltodict.parse(fd.read())

    _id = []
    wi =[]
    word = []
    lemma = []
    cpostag = []
    head = []
    deprel = []
    token_dict = {}

    conll_string = '"'
    
    conll_pattern = []
    conll_index = []
    
    for i in range(1,len(doc2['CAS']['org.apache.ctakes.typesystem.type.syntax.ConllDependencyNode'])):
        if doc2['CAS']['org.apache.ctakes.typesystem.type.syntax.ConllDependencyNode'][i]['@id'].encode("utf-8")=='0':
            continue
        token_dict[(doc2['CAS']['org.apache.ctakes.typesystem.type.syntax.ConllDependencyNode'][i]['@_id']).encode("utf-8")]=[i,(doc2['CAS']['org.apache.ctakes.typesystem.type.syntax.ConllDependencyNode'][i]['@cpostag']).encode("utf-8")]
        _id.append((doc2['CAS']['org.apache.ctakes.typesystem.type.syntax.ConllDependencyNode'][i]['@_id']).encode("utf-8"))
        wi.append((doc2['CAS']['org.apache.ctakes.typesystem.type.syntax.ConllDependencyNode'][i]['@id']).encode("utf-8"))
        word.append((doc2['CAS']['org.apache.ctakes.typesystem.type.syntax.ConllDependencyNode'][i]['@form']).encode("utf-8"))
        lemma.append((doc2['CAS']['org.apache.ctakes.typesystem.type.syntax.ConllDependencyNode'][i]['@lemma']).encode("utf-8"))
        cpostag.append((doc2['CAS']['org.apache.ctakes.typesystem.type.syntax.ConllDependencyNode'][i]['@cpostag']).encode("utf-8"))
        head.append((doc2['CAS']['org.apache.ctakes.typesystem.type.syntax.ConllDependencyNode'][i]['@_ref_head']).encode("utf-8"))
        deprel.append((doc2['CAS']['org.apache.ctakes.typesystem.type.syntax.ConllDependencyNode'][i]['@deprel']).encode("utf-8"))
        #print doc2['CAS']['org.apache.ctakes.typesystem.type.syntax.ConllDependencyNode'][i]['@form'].encode("utf-8")
        if isDrugMention((doc2['CAS']['org.apache.ctakes.typesystem.type.syntax.ConllDependencyNode'][i]['@form']).encode("utf-8")):
            conll_pattern.append('D')
            conll_index.append(int((doc2['CAS']['org.apache.ctakes.typesystem.type.syntax.ConllDependencyNode'][i]['@id']).encode("utf-8"))-1)
        
    #print "lenght:", len(doc2['CAS']['org.apache.ctakes.typesystem.type.syntax.ConllDependencyNode'])
    head_index = []
    head_pos = []
    for i in range(0,len(head)):
        try:
            head_index.append(token_dict[str(head[i])][0])
            head_pos.append(token_dict[str(head[i])][1])
        except:
            head_index.append(0)
            head_pos.append(cpostag[i])


    coNLL = {}
    for i in range(0 , len(wi)):
        coNLL[i]= {2:str(wi[i]) ,3:str(word[i]),4:str(lemma[i]),5:str(cpostag[i]),'head':str(head_index[i]),'head-pos':str(head_pos[i]),'deprel' :str(deprel[i]) }

    dev_sentence_dicts = []
    dev_sentence_dicts.append(coNLL)

    cue_ssvm, cue_vectorizer, cue_lexicon, affixal_cue_lexicon = load_cue_learner()
    dev_sd, dev_instances = negtool.extract_features_cue(dev_sentence_dicts, cue_lexicon, affixal_cue_lexicon, 'prediction')
    cue_flag = 1
    numberOfNegation = 0
    if len(dev_instances) == 0 :
        cue_flag = 0
    #print cue_flag
    if cue_flag:
        #print "Predict Cue Words"
        dev_fvs = cue_vectorizer.transform(dev_instances).toarray()
        y_pred = cue_ssvm.predict(dev_fvs)
        dev_y_pred = negtool.make_complete_labelarray(dev_sentence_dicts, y_pred)
        #print dev_y_pred

        n_mwc, has_mwc = negutil.count_multiword_cues(dev_sentence_dicts[0], dev_y_pred[0])

        

        sent_counter = 0
        line_counter = 0

        for sent_count in range(len(dev_y_pred)):
            n_cues = 0
            n_cues = sum(i > 0 for i in dev_y_pred[sent_count])
            numberOfNegation = n_cues
            n_mwc, has_mwc = negutil.count_multiword_cues(dev_sentence_dicts[sent_counter], dev_y_pred[sent_counter])
            if has_mwc:
                n_cues += n_mwc - 1
            cues = []
            mw_cues = []
            if n_cues == 0:
                dev_sentence_dicts[sent_count]['neg']=False
                dev_sentence_dicts[sent_count]['cues']=[]
                dev_sentence_dicts[sent_count]['mw_cues']=[]
            else:
                for token_count in range(len(dev_y_pred[sent_count])):
                    if dev_y_pred[sent_count][token_count]>0:
                        affix = negutil.get_affix_cue(dev_sentence_dicts[sent_count][token_count][3].lower(), affixal_cue_lexicon)
                        cue_type = 's' if affix == None else 'a'
                        prev_token = None if token_count == 0 else dev_sentence_dicts[sent_count][token_count-1][3].lower()
                        cue_token = dev_sentence_dicts[sent_count][token_count][3] if affix == None else affix
                        cue_pos = token_count
                        if negutil.mwc_start(dev_sentence_dicts[sent_count][token_count][3].lower(), prev_token):
                            cue_type = 'm'
                            mw_cues.append([cue_token, cue_pos])
                        cues.append([cue_token, cue_pos, cue_type])
                    else:
                        cue_token = dev_sentence_dicts[sent_count][token_count][3]
                        cue_pos = token_count
                        if token_count == 0:
                                prev_token = None 
                        else:
                            prev_token = dev_sentence_dicts[sent_count][token_count-1][3].lower()

                        if negutil.mwc_start(dev_sentence_dicts[sent_count][token_count][3].lower(),prev_token):
                            mw_cues.append([cue_token, token_count])
                dev_sentence_dicts[sent_count]['neg']=True
                dev_sentence_dicts[sent_count]['cues']=cues
                dev_sentence_dicts[sent_count]['mw_cues']=mw_cues

        if n_cues != 0:
            scope_ssvm, scope_vectorizer = load_scope_learner()


            sentence_dicts = dev_sentence_dicts
            try:
                sentences, dev_instances, dev_splits = negtool.extract_features_scope(sentence_dicts, 'prediction')
                dev_fvs = scope_vectorizer.transform(dev_instances).toarray()
                X_dev, y_dev = negutil.make_splits(dev_fvs, [], dev_splits)
                y_pred = scope_ssvm.predict(X_dev)
                print xmlFile
            except :
                numberOfNegation = 0
    #df = df_75[df_75['Unique_id'] == key ]
    words = ast.literal_eval(df_100['Words'][count])
    pattern =  ast.literal_eval(df_100['Pattern'][count])
    index = ast.literal_eval(df_100['Index'][count])
    
    print words, pattern, index  
    
    #print y_pred
    #print word

    new_pattern = []
    new_index = []
    new_words = []
    counter = 0
    j = 0

    neglist = []
    print conll_index
    print conll_pattern
    for j in range(0,len(conll_pattern)):
        flag = 0
        for neg in range(0,numberOfNegation):
            #print conll_index[j]
            #print y_pred
            if y_pred[neg][conll_index[j]] != 1:
                flag += 1
        if flag == 0:
            neglist.append(0)
        else:
            neglist.append(1)
    drug_count = 0
  
    for j in range(0,len(pattern)):
        if pattern[j] == 'D':
            if neglist[drug_count] == 1:
                print "check"
                #do not append
            else:
                new_pattern.append(pattern[j])
                new_index.append(index[j])
                new_words.append(words[j])                            
            drug_count+=1
        elif pattern[j] == 'N':
            print "no"
            #do not append
        else:
            new_pattern.append(pattern[j])
            new_index.append(index[j])
            new_words.append(words[j])                                
    return new_pattern , new_index , new_words , key
                    
     

In [75]:
Updated_pattern = []
Updated_words = []
Updated_index = []
Unique_id = []
count = 0
for x in range(0,len(df_100)):
    print x
    count = x 
    i = df_100['Unique_id'][x]
    pattern , index , words , key = negation_removal(i,count)
    count +=1
    Updated_pattern.append(pattern)
    Updated_words.append(words)
    Updated_index.append(index)
    Unique_id.append(key)
   

0
['Pembrolizumab', 'before'] ['D', 'O'] [16, 23]
[8, 7, 19]
['D', 'D', 'D']
1
['Pembrolizumab', 'before'] ['D', 'O'] [16, 23]
[8, 7, 19]
['D', 'D', 'D']
2
['Pembrolizumab'] ['D'] [5]
[5, 13]
['D', 'D']
3
['Pembrolizumab', 'then'] ['D', 'O'] [9, 12]
[9, 25]
['D', 'D']
4
['BRAF'] ['D'] [11]
[11, 14]
['D', 'D']
5
['Ipilimumab'] ['D'] [3]
[3, 30]
['D', 'D']
6
post_213_5.txt.xml
['Ipilimumab', 'not'] ['D', 'N'] [13, 24]
[13, 29]
['D', 'D']
no
7
['Ipilimumab'] ['D'] [13]
[13, 14]
['D', 'D']
8
['Ipilimumab'] ['D'] [6]
[6, 8]
['D', 'D']
9
['Aldesleukin'] ['D'] [18]
[18, 36]
['D', 'D']
10
['Ipilimumab', 'then', 'but'] ['D', 'O', 'N'] [5, 6, 43]
[5, 23]
['D', 'D']
no
11
['Vemurafenib'] ['D'] [26]
[26, 37]
['D', 'D']
12
['Nivolumab', 'then', 'Ipilimumab with Nivolumab', 'after'] ['D', 'O', 'D', 'O'] [5, 8, 10, 12]
[3, 5, 10, 12]
['D', 'D', 'D', 'D']
13
['Ipilimumab'] ['D'] [8]
[8, 9]
['D', 'D']
14
['MEK', 'Trametinib'] ['D', 'D'] [15, 18]
[15, 18, 20]
['D', 'D', 'D']
15
['Interferon', 'not', 'In

In [72]:
count = 137
i = df_100['Unique_id'][137]
print df_100['Tokenized.Sentences'][137]
negation_removal(i,count)

I don't know that we would have tried the Mek route had Pembrolizumab been available at that time, but it did serve its purpose very well!
post_4000_11.txt.xml
["don't", 'Mek', 'Pembrolizumab', 'but'] ['N', 'D', 'D', 'N'] [1, 10, 13, 19]
[10, 13]
['D', 'D']
no
check
check
no


([], [], [], '4000_11')

In [76]:
Updated_negtool_data = pd.DataFrame(
    {'Updated_pattern': Updated_pattern,
     'Updated_words': Updated_words,
     'Updated_index': Updated_index,
    'Unique_id':Unique_id })

In [ ]:
Updated_negtool_data.to_csv('Updated_negtool_data_v1.csv' , index=False)

In [78]:
Data_75_Updated_pattern = pd.concat([df_100,Updated_negtool_data], axis=1, join_axes=[df_100.index])

In [79]:
Data_75_Updated_pattern.to_csv('100_users_updated_negtool_v1.csv'  , index = False)

####  PatternExtraction Code

In [4]:
## Temporal extraction code

import numpy as np
import pandas as pd
import re
import nltk
import collections
import os
import tempDrugOrderNegTag as tag
from sutime import SUTime

global pattern_dict
global word_dict

jar_files = 'C:/Users/varun.behl/Documents/python-sutime-master/sutime/jars/'
sutime = SUTime(jars=jar_files,jvm_started= False , mark_time_ranges= True , include_range= True )
sutime = SUTime(jars=jar_files,jvm_started= True , mark_time_ranges= True , include_range= True )


# text = "I am taking Ipilimumab for last 10 months. Before that I was on Nivolumab from January 2015. But instead of Ipilimumab I should have taken Vemurafenib for 10 days"
# postDate = "3/7/2018  12:00:00 AM"

def get_index(typeString,replaceString,words_list,text):
    global pattern_dict
    global word_dict
    words = []
    #print type(words_list)
    for i in range(0,len(words_list)):
        print words_list[i]
        words.append(words_list[i]) 
        text = re.sub(words_list[i],replaceString ,text)
        
    z = text
    a = nltk.word_tokenize(z)
    replaceString = replaceString.split(' ')[1]
    ind = [index for index, value in enumerate(a) if value == replaceString ]
    print ind
    count = 0
    for i in ind:
        pattern_dict[i] = typeString
        word_dict[i] = words[count]
        count +=1
        
    return text

def patternExtract(text, postDate):
    global pattern_dict
    global word_dict
    pattern_dict = {}
    word_dict = {}
    
    x = sutime.parse(text , reference_date= postDate)
    
    
    Temporal = []
    Temp_pattern = []
    Temp_conversion  = []
    Temp_index  = []
    
    
    for i in range(len(x)):
        Temporal.append(x[i]['text'].encode('utf-8'))
        print Temporal
        Temp_pattern.append('T'+re.escape(str(i)))
        try:
            Temp_conversion.append(x[i]['value'].encode('utf-8'))
        except:
            Temp_conversion.append('DURATION')
    
    
    drug_words = tag.drug_tag(text)
    order_words = tag.order_words(text)
    neg_words = tag.neg_tag(text)
   


    drug_count = 0
    neg_count = 0
    order_count = 0

    text_1 = get_index('D',' DRUG ',drug_words,text)
    text_2 = get_index('O',' ORDER ',order_words,text)
    text_3 = get_index('N',' NEG ',neg_words,text)
    for i in Temporal:
        z = re.sub(i,"TEMPORAL" ,text)
        print z
        try:
            a = nltk.word_tokenize(z)
            Temp_index.append(a.index('TEMPORAL')) 
        except: 
            z = z.replace('TEMPORAL' , 'TEMPORAL ')
            a = nltk.word_tokenize(z)
            Temp_index.append(a.index('TEMPORAL'))     
    
    
    
    od_pattern = collections.OrderedDict(sorted(pattern_dict.items()))
    od_words = collections.OrderedDict(sorted(word_dict.items()))

    indices = list(od_pattern.keys())
    pattern = list(od_pattern.values())
    words = list(od_words.values())

    return pattern, indices, words , Temporal , Temp_pattern , Temp_conversion , Temp_index

# pattern, ind, words = patternExtract(text, postDate)
# print pattern
# print ind
# print words

KeyboardInterrupt: 

In [ ]:
import PatternOut as 

In [42]:
replaceString = ' DRUG '

In [44]:
replaceString.split(' ')[1]

'DRUG'

In [1]:
postDate = "2016-09-09"
text = "I have been experiencing very low HGB while on Ipilimumab since last 2 weeks."

In [ ]:
data3 = pd.read_csv('/mnt/apache-ctakes')

In [3]:
patternExtract(text ,postDate)

NameError: name 'patternExtract' is not defined